In [1]:
#pip install xgboost


Use log returns in formulas

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split,GridSearchCV
from skopt import BayesSearchCV

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tabulate import tabulate
import pandas_ta as ta
from sklearn.preprocessing import StandardScaler,MinMaxScaler,Normalizer
import statsmodels.api as sm
import itertools
from xgboost import XGBRegressor

from pandas.tseries.offsets import MonthEnd
from pandas.tseries.offsets import BDay

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows', None)

from datetime import datetime, timedelta

In [3]:
def data_loading(ticker_symbol, start_date, end_date):
    # Fetch the ETF data from Yahoo Finance for the period from 2010-01-01 to 2024-01-01
    etf_data = yf.Ticker(ticker_symbol)
    etf_history = etf_data.history(start=start_date, end=end_date)
    etf_history.index = etf_history.index.tz_localize(None)
    return etf_history, etf_data

In [4]:
#Function to derive the predictor columns
def etf_predictors(etf_history,  start_date, end_date, etf_data, benchmark_ticker = '^GSPC' ):
    
    # Calculate Daily Returns
    etf_history['Daily Return'] = etf_history['Close'].pct_change()

    # Calculate 21-Day Volatility (standard deviation of daily returns, annualized)
    etf_history['Volatility'] = etf_history['Daily Return'].rolling(window=21).std() * np.sqrt(252)
    #etf_history['Volatility_ta'] = ta.volatility(etf_history['Close'], window=21, annualize=True)
   
    # Get the ETF info
    etf_info = etf_data.info

    # Retrieve the net asset value price (NAV) and total net assets
    nav_price = etf_info['navPrice']
    total_assets = etf_info['totalAssets']

    # Calculate the number of shares outstanding
    shares_outstanding = total_assets / nav_price

    # Calculate Market Capitalization for each day
    # Market Capitalization = Closing Price * Total Number of Shares Outstanding
    etf_history['Market Cap'] = etf_history['Close'] * shares_outstanding

    total_assets = etf_info['totalAssets']
    total_liabilities = etf_info.get('totalLiabilities', 0)  # Handle the case where total liabilities might not be present
    
    # Calculate Book Value per Share
    book_value_per_share = (total_assets - total_liabilities) / shares_outstanding
    
    # Calculate Price to Book (P/B) Ratio
    etf_history['P/B Ratio'] = etf_history['Close'] / book_value_per_share
    
    # Calculate 1-Month Momentum (21 trading days)
    
    etf_history['Momentum'] = ta.mom(etf_history['Close'], length=21)
    
    benchmark_data = yf.download(benchmark_ticker, start=start_date, end=end_date)
    benchmark_data['Daily Return'] = benchmark_data['Close'].pct_change()
    #benchmark_data_history = benchmark_data.dropna()
    benchmark_data.head(10)
    
    # Ensure the indices are time zone-naive
    benchmark_data.index = benchmark_data.index.tz_localize(None)
    combined_data = etf_history[['Close']].join(benchmark_data[['Close']], lsuffix='_ETF', rsuffix='_Benchmark')
    combined_data = combined_data.dropna()
    
    # Calculate rolling beta with a 30-day window
    rolling_beta_21 = rolling_beta(combined_data, window=21)

    # Add the rolling beta to the dataframe
    combined_data.loc[:, 'Rolling Beta 21-day'] = rolling_beta_21
    etf_history['Rolling Beta']=combined_data['Rolling Beta 21-day']
    
    # Calculate daily profitability
    daily_profitabilities = []
    previous_nav = None
    for index, row in etf_history.iterrows():
        current_nav = row['Close']  # Current day's NAV
        #print(current_nav)
        if previous_nav is not None:
            daily_profitability = (current_nav - previous_nav) / previous_nav * 100
            daily_profitabilities.append(daily_profitability)
        else:
            daily_profitabilities.append(None)
        previous_nav = current_nav  # Update previous_nav for the next iteration

    # Add daily profitabilities to ETF dataset
    etf_history['Daily Profitability (%)'] = daily_profitabilities
    
    # Calculate the dividend yield for each day
    dividend_yields = []

    for index, row in etf_history.iterrows():
        # Get the dividend payment for the day
        dividend_payment = row['Dividends']

        # Get the current market price of the ETF for the day
        current_price = row['Close']

        # Calculate the dividend yield for the day
        dividend_yield = (dividend_payment / current_price) * 100

        # Append the dividend yield to the list
        dividend_yields.append(dividend_yield)
      
    etf_history['Div yield'] = dividend_yields

    #Volatility
    etf_history['ATR'] = ta.atr(etf_history['High'], etf_history['Low'], etf_history['Close'], length=21)    
    
    # Compute the Relative Volatility Index (RVI)
    rvi = ta.rvi(etf_history['Close'],length=21)
    etf_history['RVI'] = rvi
    
    #Momentum
    rsi_window = 14  # Window size for RSI calculation
    roc_window = 12  # Window size for ROC calculation
    #rsi_window = 21  
    #roc_window = 21  

    

    etf_history['RSI'] = ta.rsi(etf_history['Close'],length=rsi_window)
    # Calculate Rate of Change (ROC)
    etf_history['ROC'] = ta.roc(etf_history['Close'], length=roc_window)
        
    etf_history['log_returns'] = np.log(etf_history['Close'] / etf_history['Close'].shift(1))

    return etf_history

# Function to calculate rolling beta
def rolling_beta(df, window):
    rolling_cov = df['Close_Benchmark'].rolling(window=window).cov(df['Close_ETF'])
    rolling_var = df['Close_Benchmark'].rolling(window=window).var()
    rolling_beta = rolling_cov / rolling_var
    return rolling_beta



In [5]:
#etf_history = etf_predictors(etf_history,  start_date, end_date, etf_data,benchmark_ticker = '^GSPC' )
#etf_history.head(10)

In [6]:

def pre_processing(etf_history, train_start_date, train_end_date, test_start_date, test_end_date, prediction_dates, feature_columns=None, scaling_strategy='StandardScaler', final_end_date='2024-12-01'):
    # Handle missing values and inf replacements
    etf_history.fillna(etf_history.median(), inplace=True)
    etf_history = etf_history.replace(-np.inf, 0)

    # Default feature columns if none are provided
    if feature_columns is None:
        feature_columns = ['Volatility', 'Volume', 'Daily Return', 'Market Cap', 'P/B Ratio', 'Momentum', 
                           'Rolling Beta', 'Daily Profitability (%)', 'ATR', 'RVI', 'RSI', 'ROC']

    # Selecting the features (X) and the target (y)
    X = etf_history.loc[:, feature_columns]
    y = etf_history['log_returns']

    # Split train and test data
    train_data = X.loc[train_start_date:train_end_date]
    test_data = X.loc[test_start_date:test_end_date]
    y_train = y.loc[train_start_date:train_end_date]
    y_test = y.loc[test_start_date:test_end_date]

    # Scaling strategy based on input parameter
    if scaling_strategy == 'StandardScaler':
        scaler = StandardScaler()
    elif scaling_strategy == 'Normalizer':
        scaler = Normalizer()
    else:
        raise ValueError(f"Unsupported scaling strategy: {scaling_strategy}")
    
    # Scaling the train and test data
    train_data_scaled = scaler.fit_transform(train_data)
    test_data_scaled = scaler.transform(test_data)

    # Create a dictionary to store forecast data for each prediction period
    forecast_data = {}
    for i, start_date in enumerate(prediction_dates):
        # Determine the end date for each forecast period
        if i < len(prediction_dates) - 1:
            end_date = (pd.to_datetime(start_date) + MonthEnd(0)).strftime('%Y-%m-%d')
        else:
            end_date = final_end_date  # The final end date provided or default

        # Store the forecast data for each period
        forecast_data[f'forecast_data_{i+1}m'] = X.loc[start_date:end_date]

    # Scale the forecast data dynamically
    forecast_data_scaled = {}
    for period_key, period_data in forecast_data.items():
        if not period_data.empty:
            forecast_data_scaled[period_key] = sm.add_constant(scaler.transform(period_data))

    # Add constant to scaled train and test data
    train_data_scaled = sm.add_constant(train_data_scaled)
    test_data_scaled = sm.add_constant(test_data_scaled)

    # Dynamically return the scaled forecast data along with train and test data
    return {
        'train_data_scaled': train_data_scaled,
        'test_data_scaled': test_data_scaled,
        'y_train': y_train,
        'y_test': y_test,
        'scaler': scaler,
        'X': X,
        'etf_history': etf_history,
        'forecast_data_scaled': forecast_data_scaled,
        'forecast_data': forecast_data
    }


In [7]:
#model = sm.OLS(endog=y_train, exog=sm.add_constant(train_data_scaled))

In [8]:
def model_training(train_data_scaled, y_train):
    param_space = {
        'n_estimators': (100, 300),
        'max_depth': (3, 10),
        'learning_rate': (0.01, 0.3, 'log-uniform'),
        'gamma': (0, 0.5),
        'min_child_weight': (1, 10),
        'subsample': (0.5, 1.0),
        'colsample_bytree': (0.5, 1.0),
        'reg_alpha': (0, 1.0),
        'reg_lambda': (0, 1.0)
    }

    xgb_model = XGBRegressor(objective='reg:squarederror',random_state=42)

    bayes_search = BayesSearchCV(
        estimator=xgb_model,
        search_spaces=param_space,
        n_iter=50,
        cv=5,
        n_jobs=-1,
        verbose=0,
        scoring='neg_mean_squared_error',
        random_state=42
    )

    bayes_search.fit(train_data_scaled, y_train)
    best_model = bayes_search.best_estimator_

    print("Best Model Parameters:")
    print(bayes_search.best_params_)

    return best_model
    

In [9]:
# Display the best model parameters
#print("Best Model Parameters:")
#print(best_params)

In [10]:
def eval_model(best_model,test_data_scaled,y_test,y_train):
    
    test_predictions = best_model.predict((test_data_scaled))
    mae = mean_absolute_error(y_test, test_predictions)
    # Mean Absolute Scaled Error (MASE) - Example calculation assuming seasonal period m=1
    seasonal_naive = np.roll(y_test, 1)  # Shift y_test by 1 for seasonal naive forecast
    #seasonal_mae = np.mean(np.abs(y_test - seasonal_naive))
    seasonal_mae = np.mean(np.abs(y_test[1:] - seasonal_naive[1:]))
    mase = mae / seasonal_mae

    # Mean Absolute Percentage Error (MAPE)
    #mape = np.mean(np.abs((y_test - test_predictions) / y_test)) * 100

    #n = y_test.shape[0]
    #d = np.abs(  np.diff( y_test) ).sum()/(n-1)
    
    #errors = np.abs(y_test - test_predictions )
    #mase_value = errors.mean()/d

    print("Mean Absolute Error (MAE):", mae)
    print("Mean Absolute Scaled Error (MASE):", mase)
    #print("Mean Absolute Scaled Error-2 (MASE-2):", mase_value)
    #print("Mean Absolute Percentage Error (MAPE):", mape)
    
    return mae, mase

In [11]:
# Ensured consistent feature handling and forecasting in predictions
def predictions(model, forecast_data_scaled, forecast_data):
    predictions_forecast = model.predict(forecast_data_scaled)
    forecast_predictions_df = pd.DataFrame(predictions_forecast, columns=["log_returns"], index=forecast_data.index)
    forecast_data_with_predictions = pd.concat([forecast_data, forecast_predictions_df], axis=1)
    return forecast_predictions_df, forecast_data_with_predictions

In [12]:
#tickers = ['SPY', 'QQQ', 'VTI', 'IWM', 'EFA', 'EEM', 'GLD', 'SLV', 'USO', 'XLF']
tickers = ['SMH', 'SOXX', 'PSI', 'XSD', 'IYW', 'XLK', 'VGT', 'FTEC', 'IGM', 'IXN', 
           #'FNGU','USD', 'FNGO', 'GBTC', 'ETHE', 'TECL', 'FNGS', 'TQQQ', 'ROM', 'QLD' No data available from 2000: 2019
           ]

#tickers = ['SPY', 'QQQ', 'VTI']
#tickers = ['SPY']
                                                           
start_date, end_date = '2000-01-01', '2024-12-01'
train_start_date, train_end_date, test_start_date, test_end_date, prediction_start_date = start_date,'2014-01-01', '2014-01-01', '2024-01-01', '2024-01-01'
prediction_dates=['2024-01-01','2024-02-01','2024-03-01', '2024-04-01','2024-05-01','2024-06-01','2024-07-01','2024-08-01','2024-09-01','2024-10-01','2024-11-01']

In [13]:
dict_data = {}
etf_dict = {}
data_with_predictors = []
for i in tickers:
    etf_history,etf_data = data_loading(i, start_date, end_date)
    
    print(etf_data.info.get('longName'), ":",i)
    
    #test_stationarity(etf_history)
    print(" ETF:",i)
    #print(etf_history.head(10))
    #print("################")
    #hist_data.append(etf_history)
    
    etf_history = etf_predictors(etf_history,  start_date, end_date, etf_data,benchmark_ticker = '^GSPC' )
    #print(etf_history.head(10))
    #rolling_beta(etf_history, window)
    
    # Call the function
    preprocessed_data = pre_processing(etf_history, train_start_date, train_end_date, test_start_date, test_end_date, prediction_dates)

    # Extract individual components from the returned dictionary
    train_data_scaled = preprocessed_data['train_data_scaled']
    test_data_scaled = preprocessed_data['test_data_scaled']
    y_train = preprocessed_data['y_train']
    y_test = preprocessed_data['y_test']
    scaler = preprocessed_data['scaler']
    X = preprocessed_data['X']
    etf_history = preprocessed_data['etf_history']

    # Optionally, access forecast data for each period
    forecast_data_scaled_1m = preprocessed_data['forecast_data_scaled'].get('forecast_data_1m')
    forecast_data_scaled_2m = preprocessed_data['forecast_data_scaled'].get('forecast_data_2m')
    forecast_data_scaled_3m = preprocessed_data['forecast_data_scaled'].get('forecast_data_3m')
    forecast_data_scaled_4m = preprocessed_data['forecast_data_scaled'].get('forecast_data_4m')
    forecast_data_scaled_5m = preprocessed_data['forecast_data_scaled'].get('forecast_data_5m')
    forecast_data_scaled_6m = preprocessed_data['forecast_data_scaled'].get('forecast_data_6m')
    forecast_data_scaled_7m = preprocessed_data['forecast_data_scaled'].get('forecast_data_7m')
    
    forecast_data_scaled_8m = preprocessed_data['forecast_data_scaled'].get('forecast_data_8m')
    forecast_data_scaled_9m = preprocessed_data['forecast_data_scaled'].get('forecast_data_9m')
    forecast_data_scaled_10m = preprocessed_data['forecast_data_scaled'].get('forecast_data_10m')
    forecast_data_scaled_11m = preprocessed_data['forecast_data_scaled'].get('forecast_data_11m')
    
    
    forecast_data_1m = preprocessed_data['forecast_data'].get('forecast_data_1m')
    forecast_data_2m = preprocessed_data['forecast_data'].get('forecast_data_2m')
    forecast_data_3m = preprocessed_data['forecast_data'].get('forecast_data_3m')
    forecast_data_4m = preprocessed_data['forecast_data'].get('forecast_data_4m')
    forecast_data_5m = preprocessed_data['forecast_data'].get('forecast_data_5m')
    forecast_data_6m = preprocessed_data['forecast_data'].get('forecast_data_6m')
    forecast_data_7m = preprocessed_data['forecast_data'].get('forecast_data_7m')
    
    forecast_data_8m = preprocessed_data['forecast_data'].get('forecast_data_8m')
    forecast_data_9m = preprocessed_data['forecast_data'].get('forecast_data_9m')
    forecast_data_10m = preprocessed_data['forecast_data'].get('forecast_data_10m')
    forecast_data_11m = preprocessed_data['forecast_data'].get('forecast_data_11m')
    

    #data_with_predictors.append(etf_history)
    #print(forecast_data)
    
    best_model = model_training(train_data_scaled, y_train)
    #print(model.summary())
    
    mae, mase = eval_model(best_model, test_data_scaled, y_test, y_train)
    print(len(forecast_data_scaled_1m))
    print(len(forecast_data_scaled_2m))
    print(len(forecast_data_scaled_3m))
    print(len(forecast_data_scaled_4m))
    print(len(forecast_data_scaled_5m))
    print(len(forecast_data_scaled_6m))
    print(len(forecast_data_scaled_7m))
    
    print(len(forecast_data_scaled_8m))
    print(len(forecast_data_scaled_9m))
    print(len(forecast_data_scaled_10m))
    print(len(forecast_data_scaled_11m))
    
    
    forecast_predictions_df_1m, forecast_data_with_predictions_1m = predictions(best_model, forecast_data_scaled_1m, forecast_data_1m)
    forecast_predictions_df_2m, forecast_data_with_predictions_2m = predictions(best_model, forecast_data_scaled_2m, forecast_data_2m)
    forecast_predictions_df_3m, forecast_data_with_predictions_3m = predictions(best_model, forecast_data_scaled_3m, forecast_data_3m)
    forecast_predictions_df_4m, forecast_data_with_predictions_4m = predictions(best_model, forecast_data_scaled_4m, forecast_data_4m)
    forecast_predictions_df_5m, forecast_data_with_predictions_5m = predictions(best_model, forecast_data_scaled_5m, forecast_data_5m)
    forecast_predictions_df_6m, forecast_data_with_predictions_6m = predictions(best_model, forecast_data_scaled_6m, forecast_data_6m)
    forecast_predictions_df_7m, forecast_data_with_predictions_7m = predictions(best_model, forecast_data_scaled_7m, forecast_data_7m)
    
    forecast_predictions_df_8m, forecast_data_with_predictions_8m = predictions(best_model, forecast_data_scaled_8m, forecast_data_8m)
    forecast_predictions_df_9m, forecast_data_with_predictions_9m = predictions(best_model, forecast_data_scaled_9m, forecast_data_9m)
    forecast_predictions_df_10m, forecast_data_with_predictions_10m = predictions(best_model, forecast_data_scaled_10m, forecast_data_10m)
    forecast_predictions_df_11m, forecast_data_with_predictions_11m = predictions(best_model, forecast_data_scaled_11m, forecast_data_11m)
    
    
    # Add data to dictionary
    # Store the relevant data in a dictionary

    #print(forecast_predictions_df_6m)
    #print(forecast_predictions_df_7m)
    
    dict_data = {
        "etf_history": etf_history,
        "forecast_data_1m": forecast_data_1m,
        "forecast_data_2m": forecast_data_2m,
        "forecast_data_3m": forecast_data_3m,
        "forecast_data_4m": forecast_data_4m,
        "forecast_data_5m": forecast_data_5m,
        "forecast_data_6m": forecast_data_6m,
        "forecast_data_7m": forecast_data_7m,
        
        "forecast_data_8m": forecast_data_8m,
        "forecast_data_9m": forecast_data_9m,
        "forecast_data_10m": forecast_data_10m,
        "forecast_data_11m": forecast_data_11m,
        
        "X": X,
        "y_train_values": y_train , # Assuming this is relevant to the ETF data
        "model results": {"mae": mae,
                          "mase": mase,
                          #"mape": mape
                         },
                         
        "forecast_predictions_df_1m" : forecast_predictions_df_1m['log_returns'].values,
        "forecast_predictions_df_2m" : forecast_predictions_df_2m['log_returns'].values,
        "forecast_predictions_df_3m" : forecast_predictions_df_3m['log_returns'].values,
        "forecast_predictions_df_4m" : forecast_predictions_df_4m['log_returns'].values,
        "forecast_predictions_df_5m" : forecast_predictions_df_5m['log_returns'].values,
        "forecast_predictions_df_6m" : forecast_predictions_df_6m['log_returns'].values,
        "forecast_predictions_df_7m" : forecast_predictions_df_7m['log_returns'].values,
        
        "forecast_predictions_df_8m" : forecast_predictions_df_8m['log_returns'].values,
        "forecast_predictions_df_9m" : forecast_predictions_df_9m['log_returns'].values,
        "forecast_predictions_df_10m" : forecast_predictions_df_10m['log_returns'].values,
        "forecast_predictions_df_11m" : forecast_predictions_df_11m['log_returns'].values
    }
    
    # Associate the dictionary with the ETF ticker symbol
    etf_dict[i] = dict_data
     

VanEck Semiconductor ETF : SMH
 ETF: SMH


[*********************100%%**********************]  1 of 1 completed


Best Model Parameters:
OrderedDict([('colsample_bytree', 0.6378182931238394), ('gamma', 0.0), ('learning_rate', 0.29999999999999993), ('max_depth', 5), ('min_child_weight', 2), ('n_estimators', 242), ('reg_alpha', 0.0), ('reg_lambda', 0.2201436318350558), ('subsample', 0.9191126360221957)])
Mean Absolute Error (MAE): 0.00029819338248419187
Mean Absolute Scaled Error (MASE): 0.015467296924389622
21
20
20
22
22
19
22
22
20
23
20


[*********************100%%**********************]  1 of 1 completed

iShares Semiconductor ETF : SOXX
 ETF: SOXX


...

iShares Expanded Tech Sector ETF : IGM
 ETF: IGM


Best Model Parameters:
OrderedDict([('colsample_bytree', 0.6261163620689146), ('gamma', 0.0), ('learning_rate', 0.29999999999999993), ('max_depth', 10), ('min_child_weight', 9), ('n_estimators', 157), ('reg_alpha', 0.0), ('reg_lambda', 0.3965278139782738), ('subsample', 0.9147007080545038)])
Mean Absolute Error (MAE): 0.0012759013904846104
Mean Absolute Scaled Error (MASE): 0.08748615069823613
21
20
20
22
22
19
22
22
20
23
20


[*********************100%%**********************]  1 of 1 completed

iShares Global Tech ETF : IXN
 ETF: IXN


Best Model Parameters:
OrderedDict([('colsample_bytree', 0.600882345199172), ('gamma', 0.0), ('learning_rate', 0.21826279406943389), ('max_depth', 3), ('min_child_weight', 2), ('n_estimators', 297), ('reg_alpha', 0.0), ('reg_lambda', 0.6005489733469533), ('subsample', 1.0)])
Mean Absolute Error (MAE): 0.0005127583174306603
Mean Absolute Scaled Error (MASE): 0.03617775012985391
21
20
20
22
22
19
22
22
20
23
20


In [27]:
def calculate_sharpe_ratio(returns, annual_risk_free_rate=0.1,period='daily'):
    #excess_returns = rate_of_return(returns) - risk_free_rate
    
    # Convert annual risk-free rate to daily rate
    daily_risk_free_rate = (1 + annual_risk_free_rate) ** (1/252) - 1
    
    # Calculate mean daily log return
    mean_return = np.mean(returns)
    
    # Calculate excess daily log return
    excess_return = mean_return - daily_risk_free_rate
    
    # Calculate standard deviation of daily log returns
    std_return = np.std(returns)
    
    # Print diagnostic information
    #print(f"Mean Daily Log Return: {mean_return}")
    #print(f"Excess Daily Log Return: {excess_return}")
    #print(f"Standard Deviation of Daily Log Returns: {std_return}")
    
    # Check for zero standard deviation to avoid division by zero
    if std_return == 0:
        return 0
    
    # Calculate Sharpe ratio
    sharpe_ratio = (excess_return / std_return) * np.sqrt(252)  # Annualize the Sharpe ratio
    return sharpe_ratio
    #return excess_returns / np.std(returns)


def calculate_rachev_ratio(returns, lower_percentile=5, upper_percentile=95):
    # Step 1: Sort the returns
    sorted_returns = np.sort(returns)
    
    # Step 2: Determine the percentiles
    lower_threshold = np.percentile(sorted_returns, lower_percentile)
    upper_threshold = np.percentile(sorted_returns, upper_percentile)
    
    # Step 3: Calculate Expected Shortfall (ES)
    es = np.mean(sorted_returns[sorted_returns <= lower_threshold])
    
    # Step 4: Calculate Expected Gain (EG)
    eg = np.mean(sorted_returns[sorted_returns >= upper_threshold])
    
    # Step 5: Compute the Rachev Ratio
    rachev_ratio = eg / -es
    return rachev_ratio


def calculate_volatility_clustering(returns):
    squared_returns = returns ** 2
    n = len(squared_returns)
    
    # Mean of squared returns
    mean_squared_returns = np.mean(squared_returns)
    
    # Calculate the numerator and denominator for autocorrelation at lag 1
    numerator = np.sum((squared_returns[:-1] - mean_squared_returns) * (squared_returns[1:] - mean_squared_returns))
    denominator = np.sum((squared_returns - mean_squared_returns) ** 2)
    
    if denominator == 0:
        return 0  # Avoid division by zero
    
    rho_1 = numerator / denominator
    return rho_1

def calculate_sortino_ratio(log_returns, target_log_return=0.0):
    """
    Calculate the Sortino Ratio using log returns.
    
    Parameters:
    - log_returns (array-like): Array or list of log returns for the period.
    - target_log_return (float): The target log return. Default is 0, which is often used as a benchmark.
    
    Returns:
    - float: The Sortino Ratio.
    """
    # Convert input to a NumPy array for easier calculations
    log_returns = np.array(log_returns)
    
    # Calculate the average period log return (R)
    avg_log_return = np.mean(log_returns)
    
    # Calculate the Target Downside Deviation (TDD)
    downside_deviation = np.sqrt(
        np.mean(np.square(np.maximum(0, target_log_return - log_returns)))
    )
    
    # Calculate Sortino Ratio
    #sortino_ratio_value = (avg_log_return - target_log_return) / downside_deviation
    epsilon = 1e-8
    
    # Add epsilon to downside_deviation to prevent division by zero
    #if downside_deviation == 0:
     #   return np.nan  # Return NaN if downside deviation is zero
    
    # Calculate Sortino Ratio
    sortino_ratio_value = (avg_log_return - target_log_return) / (downside_deviation + epsilon)
    
    return sortino_ratio_value


In [28]:
def calculate_composite_score(
    forecasted_values, risk_percentage, rachev_ratio, sharpe_ratio, sortino_ratio, volatility_clustering, 
    mean_forecast, std_forecast, mean_rachev, std_rachev, mean_sharpe, std_sharpe, mean_sortino, std_sortino, 
    mean_volatility_clustering, std_volatility_clustering
):
    forecasted_mean = np.mean(forecasted_values)
    forecasted_mean_normalized = (forecasted_mean - mean_forecast) / std_forecast
    rachev_normalized = (rachev_ratio - mean_rachev) / std_rachev
    sharpe_normalized = (sharpe_ratio - mean_sharpe) / std_sharpe
    
    sortino_normalized = (sortino_ratio - mean_sortino) / std_sortino
    
    volatility_clustering_normalized = (volatility_clustering - mean_volatility_clustering) / std_volatility_clustering
    
    score = forecasted_mean_normalized - (risk_percentage * rachev_normalized) + sharpe_normalized + sortino_normalized - volatility_clustering_normalized
    return score


In [40]:
def calculate_composite_score(
    forecasted_values, risk_percentage, rachev_ratio, sharpe_ratio, sortino_ratio, volatility_clustering, 
    mean_forecast, std_forecast, mean_rachev, std_rachev, mean_sharpe, std_sharpe, mean_sortino, std_sortino, 
    mean_volatility_clustering, std_volatility_clustering
):
    epsilon = 1e-8  # To prevent division by zero
    forecasted_mean = np.mean(forecasted_values)

    # Debugging: Log raw input values
    print(f"\nDebug: Composite Score Calculation")
    print(f"Forecasted Mean: {forecasted_mean}, Risk Percentage: {risk_percentage}")
    print(f"Rachev Ratio: {rachev_ratio}, Sharpe Ratio: {sharpe_ratio}")
    print(f"Sortino Ratio: {sortino_ratio}, Volatility Clustering: {volatility_clustering}")

    # Normalize the components with epsilon
    forecasted_mean_normalized = (forecasted_mean - mean_forecast) / (std_forecast + epsilon)
    rachev_normalized = (rachev_ratio - mean_rachev) / (std_rachev + epsilon)
    sharpe_normalized = (sharpe_ratio - mean_sharpe) / (std_sharpe + epsilon)
    sortino_normalized = (sortino_ratio - mean_sortino) / (std_sortino + epsilon)
    volatility_clustering_normalized = (volatility_clustering - mean_volatility_clustering) / (std_volatility_clustering + epsilon)

    # Debugging: Log normalized values
    print(f"Normalized Values -> Forecasted Mean: {forecasted_mean_normalized}, Rachev: {rachev_normalized}")
    print(f"Sharpe: {sharpe_normalized}, Sortino: {sortino_normalized}, Volatility Clustering: {volatility_clustering_normalized}")

    # Composite score calculation
    score = (
        forecasted_mean_normalized
        - (risk_percentage * rachev_normalized)
        + sharpe_normalized
        + sortino_normalized
        - volatility_clustering_normalized
    )

    # Debugging: Log final score
    print(f"Final Composite Score: {score}")

    return score


def process_etf_data_monthly(tickers, etf_dict):
    etf_pred_dict = {}

    # Determine the forecast periods dynamically from the etf_dict
    sample_etf = next(iter(etf_dict.values()))
    forecast_periods = [key.split('_')[-1] for key in sample_etf.keys() if key.startswith('forecast_predictions_df')]

    for etf_name in tickers:
        etf_pred_dict[etf_name] = {
            f"returns_{period}": etf_dict[etf_name][f"forecast_predictions_df_{period}"]
            for period in forecast_periods
        }

        for period in forecast_periods:
            returns = etf_pred_dict[etf_name][f"returns_{period}"]

            # Log returns for debugging
            print(f"\nDebug: Returns for {etf_name}, Forecast Period {period}: {returns}")

            # Calculate metrics
            etf_pred_dict[etf_name][f"rachev_ratio_{period}"] = calculate_rachev_ratio(returns)
            etf_pred_dict[etf_name][f"sharpe_ratio_{period}"] = calculate_sharpe_ratio(returns)
            etf_pred_dict[etf_name][f"sortino_ratio_{period}"] = calculate_sortino_ratio(returns)
            etf_pred_dict[etf_name][f"volatility_clustering_{period}"] = calculate_volatility_clustering(returns)

    return etf_pred_dict


def calculate_means_and_stds_monthly(etf_pred_dict, forecast_period):
    returns_list = [etf_pred_dict[etf][f'returns_{forecast_period}'] for etf in etf_pred_dict]

    # Compute global means and standard deviations
    mean_forecast = np.mean([np.mean(returns) for returns in returns_list])
    std_forecast = np.std([np.mean(returns) for returns in returns_list])

    print(f"\nDebug: Forecast Period = {forecast_period}")
    print(f"All Returns Means: {[np.mean(returns) for returns in returns_list]}")
    print(f"Mean Forecast = {mean_forecast}, Std Forecast = {std_forecast}")

    rachev_ratios = np.array([etf_pred_dict[etf][f'rachev_ratio_{forecast_period}'] for etf in etf_pred_dict])
    print(f"All Rachev Ratios: {rachev_ratios}")
    mean_rachev = np.mean(rachev_ratios)
    std_rachev = np.std(rachev_ratios)

    sharpe_ratios = np.array([etf_pred_dict[etf][f'sharpe_ratio_{forecast_period}'] for etf in etf_pred_dict])
    print(f"All Sharpe Ratios: {sharpe_ratios}")
    mean_sharpe = np.mean(sharpe_ratios)
    std_sharpe = np.std(sharpe_ratios)

    sortino_ratios = np.array([etf_pred_dict[etf][f'sortino_ratio_{forecast_period}'] for etf in etf_pred_dict])
    print(f"All Sortino Ratios: {sortino_ratios}")
    mean_sortino = np.mean(sortino_ratios)
    std_sortino = np.std(sortino_ratios)

    volatility_clustering = np.array([etf_pred_dict[etf][f'volatility_clustering_{forecast_period}'] for etf in etf_pred_dict])
    print(f"All Volatility Clustering: {volatility_clustering}")
    mean_volatility_clustering = np.mean(volatility_clustering)
    std_volatility_clustering = np.std(volatility_clustering)

    print(f"Mean Rachev = {mean_rachev}, Std Rachev = {std_rachev}")
    print(f"Mean Sharpe = {mean_sharpe}, Std Sharpe = {std_sharpe}")
    print(f"Mean Sortino = {mean_sortino}, Std Sortino = {std_sortino}")
    print(f"Mean Volatility Clustering = {mean_volatility_clustering}, Std Volatility Clustering = {std_volatility_clustering}")

    return (
        mean_forecast, std_forecast, mean_rachev, std_rachev,
        mean_sharpe, std_sharpe, mean_sortino, std_sortino,
        mean_volatility_clustering, std_volatility_clustering
    )


def calculate_scores_for_etfs_monthly(etf_pred_dict, forecast_period, risk_percentage):
    (mean_forecast, std_forecast, mean_rachev, std_rachev, 
     mean_sharpe, std_sharpe, mean_sortino, std_sortino, 
     mean_volatility_clustering, std_volatility_clustering) = calculate_means_and_stds_monthly(etf_pred_dict, forecast_period)

    scores = []
    for etf in etf_pred_dict:
        forecasted_values = etf_pred_dict[etf][f'returns_{forecast_period}']
        rachev_ratio = etf_pred_dict[etf][f'rachev_ratio_{forecast_period}']
        sharpe_ratio = etf_pred_dict[etf][f'sharpe_ratio_{forecast_period}']
        volatility_clustering = etf_pred_dict[etf][f'volatility_clustering_{forecast_period}']
        sortino_ratio = etf_pred_dict[etf][f'sortino_ratio_{forecast_period}']

        # Debugging: Log inputs to composite score calculation
        print(f"\nDebug: ETF = {etf}, Forecast Period = {forecast_period}")
        print(f"Forecasted Values Mean: {np.mean(forecasted_values)}")
        print(f"Rachev Ratio: {rachev_ratio}, Sharpe Ratio: {sharpe_ratio}")
        print(f"Sortino Ratio: {sortino_ratio}, Volatility Clustering: {volatility_clustering}")
        print(f"Means and Stds: Mean Forecast = {mean_forecast}, Std Forecast = {std_forecast}")
        
        # Calculate the composite score
        score = calculate_composite_score(
            forecasted_values, risk_percentage, rachev_ratio, sharpe_ratio, 
            sortino_ratio, volatility_clustering, mean_forecast, std_forecast, 
            mean_rachev, std_rachev, mean_sharpe, std_sharpe, mean_sortino, 
            std_sortino, mean_volatility_clustering, std_volatility_clustering
        )

        scores.append({
            'ETF': etf,
            'Month': forecast_period,
            'RiskPercentage': risk_percentage,
            'Score': score
        })

        # Debugging: Log the calculated score
        print(f"Calculated Score for {etf} ({forecast_period}): {score}")

    return scores


def main_monthly(tickers, etf_dict):
    etf_pred_dict = process_etf_data_monthly(tickers, etf_dict)
    
    risk_percentage = 0.10
    monthly_scores = {}

    # Iterate over 12 months
    for month in range(1, 13):
        month_key = f"{month}m"
        if any(f"returns_{month_key}" in etf_pred_dict[etf] for etf in etf_pred_dict):
            scores = calculate_scores_for_etfs_monthly(etf_pred_dict, month_key, risk_percentage)
            monthly_scores[month_key] = scores
            print(f"Scores calculated for {month_key}:")
            for score in scores:
                print(score)
        else:
            print(f"Skipping score calculation for {month_key}: No ETFs have data for this month.")

    return etf_pred_dict, monthly_scores


# Example usage:
# Execute monthly scoring
etf_pred_dict, monthly_scores = main_monthly(tickers, etf_dict)





Debug: Returns for SMH, Forecast Period 1m: [-0.03424207 -0.0177656  -0.00613295  0.00685085  0.03585814  0.00294293
  0.00110846  0.00592081 -0.00325445  0.01588209 -0.0060796   0.03215366
  0.03768135  0.00474041  0.00557297  0.0177254   0.00030171 -0.02152142
  0.01216641 -0.00953063 -0.0142531 ]

Debug: Returns for SMH, Forecast Period 2m: [ 0.00914876  0.01957979  0.01957979 -0.01109389  0.0199053   0.01432536
  0.022254   -0.0037174  -0.01843396  0.0199053  -0.00325445 -0.00413795
 -0.01976739 -0.00620238  0.06652507 -0.00613295  0.00850707 -0.00252586
 -0.01108421  0.02176784]

Debug: Returns for SMH, Forecast Period 3m: [ 4.0259127e-02  1.6403226e-02 -1.4875734e-02  2.5091643e-02
  3.4783494e-02 -3.9856419e-02 -1.7832287e-02  3.1111727e-02
 -1.9767394e-02 -1.7912013e-02 -6.9202459e-03  2.1317771e-03
 -2.0153981e-03  1.6323499e-02  2.2416493e-02  6.1173928e-03
 -2.2397200e-03 -9.1863824e-03 -8.0723556e-05  8.6368917e-04]

Debug: Returns for SMH, Forecast Period 4m: [ 0.01366763

In [41]:
# Initialize an empty dictionary to hold DataFrames for each month
monthly_scores_dfs = {}

# Assuming monthly_scores is a dictionary with keys as month identifiers ('1m', '2m', ..., '12m') 
# and values as the respective scores
for month_key, scores in monthly_scores.items():
    # Convert scores for the current month into a DataFrame
    monthly_scores_dfs[month_key] = pd.DataFrame(scores)

    
# Create DataFrame variables dynamically for 12 months
for month in range(1, 13):
    month_key = f'{month}m'
    globals()[f'df_scores_{month_key}'] = monthly_scores_dfs.get(month_key, pd.DataFrame())




In [42]:
# Initialize a dictionary to hold the top 2 ETFs for each month
top_etfs_monthly = {}

# Loop through the monthly DataFrames and select the top 2 ETFs for each month
for month_key, df_scores in monthly_scores_dfs.items():
    # Select the top 2 ETFs based on the 'Score' column
    top_etfs_monthly[month_key] = df_scores.nlargest(2, 'Score')
    
    # Print the results for the current month
    print(f"\nTop 2 ETFs for {month_key} forecast:")
    print(top_etfs_monthly[month_key])


Top 2 ETFs for 1m forecast:
    ETF Month  RiskPercentage     Score
7  FTEC    1m             0.1  3.669785
5   XLK    1m             0.1  1.092880

Top 2 ETFs for 2m forecast:
    ETF Month  RiskPercentage     Score
7  FTEC    2m             0.1  2.200025
0   SMH    2m             0.1  1.055449

Top 2 ETFs for 3m forecast:
    ETF Month  RiskPercentage     Score
7  FTEC    3m             0.1  3.662326
5   XLK    3m             0.1  1.224005

Top 2 ETFs for 4m forecast:
    ETF Month  RiskPercentage     Score
7  FTEC    4m             0.1  5.524037
5   XLK    4m             0.1  0.681113

Top 2 ETFs for 5m forecast:
    ETF Month  RiskPercentage     Score
7  FTEC    5m             0.1  1.738725
5   XLK    5m             0.1  1.242787

Top 2 ETFs for 6m forecast:
    ETF Month  RiskPercentage     Score
7  FTEC    6m             0.1  2.512000
4   IYW    6m             0.1  0.979679

Top 2 ETFs for 7m forecast:
    ETF Month  RiskPercentage     Score
7  FTEC    7m             0.1  5.0058

In [43]:
def select_top_etfs_monthly(df_scores, forecast_period):
    if df_scores.empty:
        print(f"No scores available for {forecast_period}. Skipping.")
        return []
    print(f"Processing scores for {forecast_period}:")
    print(df_scores.head())  # Check the top rows of the DataFrame
    top_etfs = df_scores.nlargest(2, 'Score')
    print(f"Top ETFs for {forecast_period}: {top_etfs['ETF'].tolist()}")
    return top_etfs['ETF'].tolist()

# Function to generate month ranges
def generate_month_ranges(start_date, end_date):
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    month_ranges = []

    while start < end:
        # Start of the current month
        month_start = start
        # End of the current month
        next_month_start = (start + timedelta(days=32)).replace(day=1)
        month_end = next_month_start - timedelta(days=1)

        # Ensure the end date doesn't go beyond the specified end_date
        if month_end > end:
            month_end = end

        # Add the range to the list
        month_ranges.append((month_start.strftime('%Y-%m-%d'), month_end.strftime('%Y-%m-%d')))
        # Move to the next month's start
        start = next_month_start

    return month_ranges

# Function to gather ETF data for months
def gather_etf_data_for_months(tickers, month_ranges):
    etf_histories = {}
    for start_date, end_date in month_ranges:
        month = f"{start_date} to {end_date}"
        etf_histories[month] = {}
        for ticker in tickers:
            etf_data = yf.download(ticker, start=start_date, end=end_date)
            if etf_data.empty:
                print(f"No data found for {ticker} in {month}")
                continue
            etf_data.index = pd.to_datetime(etf_data.index)
            etf_histories[month][ticker] = etf_data
    return etf_histories

# Function to initialize shares for the first month
def initialize_shares_for_first_month(top_etfs_1m, etf_histories, month, investment_amount=50000):
    ticker_shares = {}
    first_trading_day_start = month.split(" to ")[0]

    for ticker in top_etfs_1m:
        etf_history = etf_histories.get(month, {}).get(ticker)

        if etf_history is not None:
            first_trading_day = pd.to_datetime(first_trading_day_start) + BDay(1)
            if first_trading_day not in etf_history.index:
                first_trading_day = etf_history.index[etf_history.index.searchsorted(first_trading_day)]

            price_on_first_trading_day = etf_history.loc[first_trading_day, 'Close']
            num_shares = (investment_amount * 0.975) / price_on_first_trading_day
            print(f"Shares 1st month: ({investment_amount} * 0.975) / {price_on_first_trading_day}")
            ticker_shares[ticker] = num_shares
            print(f"Ticker: {ticker}, First trading day: {first_trading_day.date()}, Price: {price_on_first_trading_day}, Shares: {num_shares:.2f}")
        else:
            print(f"No data found for {ticker} in {month}")
    return ticker_shares

# Function to manage ETF portfolio monthly
def manage_etf_portfolio_monthly(
    top_etfs_previous, top_etfs_current, previous_month, current_month, ticker_shares, gathered_data_per_month
):
    etf_histories_for_current_month = gathered_data_per_month.get(current_month, {})

    # Extract ETFs as lists to preserve order
    top2etfs_previous = list(top_etfs_previous)
    top2etfs_current = list(top_etfs_current)

    print(f"Top 2 ETFs for {previous_month}: {top2etfs_previous}")
    print(f"Top 2 ETFs for {current_month}: {top2etfs_current}")

    # Step 1: Identify common ETFs (present in both months)
    etfs_common = [etf for etf in top2etfs_previous if etf in top2etfs_current]

    # Step 2: Identify ETFs to sell (present in the previous month but not in the current month)
    etfs_to_sell = [etf for etf in top2etfs_previous if etf not in top2etfs_current]

    # Step 3: Identify ETFs to buy (present in the current month but not in the previous month)
    etfs_to_buy = [etf for etf in top2etfs_current if etf not in top2etfs_previous]

    # Step 4: Sell ETFs that are no longer in the current top ETFs
    total_selling_value = 0
    for etf in etfs_to_sell:
        no_of_shares = ticker_shares.get(etf, 0)
        if no_of_shares > 0:
            # Check if data for the ETF is available in the current month
            if etf in etf_histories_for_current_month:
                first_trading_day_current_month = etf_histories_for_current_month[etf].loc[etf_histories_for_current_month[etf].index[0], 'Close']
                
                selling_value = no_of_shares * first_trading_day_current_month * 0.975
                print("Formula: ",no_of_shares," * ", first_trading_day_current_month,"* 0.975")
                total_selling_value += selling_value  # Sum total selling value
                print(f"Sell {etf}: {no_of_shares:.2f} shares at {first_trading_day_current_month:.2f}. Total value: {selling_value:.2f}  \n")
                del ticker_shares[etf]
            else:
                print(f"Data for {etf} is missing for {current_month}. Skipping sale.")
        else:
            print(f"No shares found for {etf} to sell.")

    # Step 5: Buy new ETFs that were not in the previous month's top ETFs
    for etf in etfs_to_buy:
        if total_selling_value > 0 and etf in etf_histories_for_current_month:
            first_trading_day_new_etf = etf_histories_for_current_month[etf].loc[etf_histories_for_current_month[etf].index[0], 'Close']
            new_shares = (total_selling_value * 0.975) / first_trading_day_new_etf
            print("Formula: ()",total_selling_value," * 0.975)/", first_trading_day_current_month)
            
            print(f"Buy {etf}: {new_shares:.2f} shares at {first_trading_day_new_etf:.2f}. \n")
            ticker_shares[etf] = new_shares
        else:
            print(f"Data for {etf} is missing or no selling value available. Skipping purchase of {etf}.")

    # Step 6: Maintain the order of ETFs in the ticker_shares dictionary based on the current top 2 ETFs
    ordered_ticker_shares = {etf: ticker_shares[etf] for etf in top2etfs_current if etf in ticker_shares}

    print(f"Updated ticker shares after {current_month}: {ordered_ticker_shares}")
    print("")
    return ordered_ticker_shares



In [44]:
# Generate Month Ranges
month_ranges = generate_month_ranges('2024-01-01', '2024-12-01')

# Assuming `tickers` is a list of ETF tickers
tickers = tickers

# Gather ETF Data
etf_histories = gather_etf_data_for_months(tickers, month_ranges)

# Create a mapping between month numbers and date ranges
month_key_mapping = {f"{i+1}m": month_range for i, month_range in enumerate(etf_histories.keys())}

# Debug: Print the month key mapping
print("Month Key Mapping:", month_key_mapping)

# Align `top_etfs_monthly` Keys with `etf_histories`
aligned_top_etfs_monthly = {}

for month_key, df_scores in monthly_scores_dfs.items():
    # Call the `select_top_etfs_monthly` function here
    forecast_period = month_key_mapping.get(month_key, None)
    if forecast_period:
        aligned_top_etfs_monthly[forecast_period] = select_top_etfs_monthly(df_scores, forecast_period)

# Portfolio initialization and management
ticker_shares = {}
ticker_shares_per_month = {}

for i, month_range in enumerate(etf_histories.keys()):
    current_month_key = month_range
    if i == 0:
        # First month initialization
        ticker_shares = initialize_shares_for_first_month(
            aligned_top_etfs_monthly[current_month_key],
            etf_histories,
            current_month_key
        )
        ticker_shares_per_month[current_month_key] = ticker_shares.copy()
    else:
        # Subsequent months' portfolio management
        prev_month_key = list(etf_histories.keys())[i - 1]
        ticker_shares = manage_etf_portfolio_monthly(
            aligned_top_etfs_monthly[prev_month_key],
            aligned_top_etfs_monthly[current_month_key],
            prev_month_key,
            current_month_key,
            ticker_shares,
            etf_histories
        )
        ticker_shares_per_month[current_month_key] = ticker_shares.copy()

print("\nFinal Ticker Shares per Month:")
for month, shares in ticker_shares_per_month.items():
    print(f"Month {month}: {shares}")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Month Key Mapping: {'1m': '2024-01-01 to 2024-01-31', '2m': '2024-02-01 to 2024-02-29', '3m': '2024-03-01 to 2024-03-31', '4m': '2024-04-01 to 2024-04-30', '5m': '2024-05-01 to 2024-05-31', '6m': '2024-06-01 to 2024-06-30', '7m': '2024-07-01 to 2024-07-31', '8m': '2024-08-01 to 2024-08-31', '9m': '2024-09-01 to 2024-09-30', '10m': '2024-10-01 to 2024-10-31', '11m': '2024-11-01 to 2024-11-30'}
Processing scores for 2024-01-01 to 2024-01-31:
    ETF Month  RiskPercentage     Score
0   SMH    1m             0.1  0.428210
1  SOXX    1m             0.1 -0.789703
2   PSI    1m             0.1 -1.173371
3   XSD    1m             0.1 -3.328748
4   IYW    1m             0.1  1.027073
Top ETFs for 2024-01-01 to 2024-01-31: ['FTEC', 'XLK']
Processing scores for 2024-02-01 to 2024-02-29:
    ETF Month  RiskPercentage     Score
0   SMH    2m             0.1  1.055449
1  SOXX    2m             0.1  0.519766
2   PSI    2m             0.1  0.979487
3   XSD    2m             0.1 -2.091709
4   IYW    2m

In [45]:
# Define the first trading day of December 2024
first_trading_day_dec = '2024-12-01'  # Adjust this to match the actual start date of December

# Identify the November and December month date range keys
november_range = list(ticker_shares_per_month.keys())[-1]  # Last key corresponds to November
december_start = first_trading_day_dec  # Replace with the actual start of December
print(f"Using data for November: {november_range}")

print(f"Fetching data starting from the first trading day of December: {december_start}")

december_end = '2024-12-31'
# Initialize a dictionary to store the values of shares
etf_values_dec = {}

# Ensure November data exists
if november_range in ticker_shares_per_month:
    # Fetch ETF shares from November
    ticker_shares_nov = ticker_shares_per_month[november_range]
    
    # Fetch the first trading day price of December for each ETF
    for ticker, shares in ticker_shares_nov.items():
        print(f"Fetching data for ticker {ticker} starting from {december_start}...")
        # Download historical data for December
        data = yf.download(ticker, start=december_start, end=december_end)
        
        if not data.empty:
            # Get the closing price of the first trading day of December
            closing_price_dec = data['Close'].iloc[0]
            # Calculate the value of the shares
            total_value = shares * closing_price_dec
            etf_values_dec[ticker] = total_value
            print(f"{ticker}: {shares:.2f} shares at ${closing_price_dec:.2f} each, total value: ${total_value:.2f}")
        else:
            print(f"{ticker}: No data available for December's first trading day.")
else:
    print(f"No data available in ticker_shares_per_month for November: {november_range}")



[*********************100%%**********************]  1 of 1 completed


Using data for November: 2024-11-01 to 2024-11-30
Fetching data starting from the first trading day of December: 2024-12-01
Fetching data for ticker FTEC starting from 2024-12-01...
FTEC: 348.74 shares at $186.99 each, total value: $65210.41
Fetching data for ticker XSD starting from 2024-12-01...


[*********************100%%**********************]  1 of 1 completed

XSD: 218.58 shares at $252.68 each, total value: $55229.59


### Values for 11th Month

In [46]:
# Check if there are any values in the dictionary
if etf_values_dec:
    print("\nETF values on December's first trading day:")
    # Sum all the values (total portfolio value)
    total_value = sum(etf_values_dec.values())
    print(f"Total portfolio value: {total_value:.2f}")
    # Optionally display individual ETF values
    for ticker, value in etf_values_dec.items():
        print(f"{ticker}: {value:.2f}")
else:
    print("No values could be calculated for December's first trading day.")



ETF values on December's first trading day:
Total portfolio value: 120440.00
FTEC: 65210.41
XSD: 55229.59


### Smoothing

In [47]:
def calculate_composite_score(
    forecasted_values, risk_percentage, rachev_ratio, sharpe_ratio, sortino_ratio, 
    volatility_clustering, mean_forecast, std_forecast, mean_rachev, std_rachev, 
    mean_sharpe, std_sharpe, mean_sortino, std_sortino, mean_volatility_clustering, 
    std_volatility_clustering
):
    epsilon = 1e-8  # To prevent division by zero
    forecasted_mean = np.mean(forecasted_values)

    forecasted_mean_normalized = (forecasted_mean - mean_forecast) / (std_forecast + epsilon)
    rachev_normalized = (rachev_ratio - mean_rachev) / (std_rachev + epsilon)
    sharpe_normalized = (sharpe_ratio - mean_sharpe) / (std_sharpe + epsilon)
    sortino_normalized = (sortino_ratio - mean_sortino) / (std_sortino + epsilon)
    volatility_clustering_normalized = (volatility_clustering - mean_volatility_clustering) / (std_volatility_clustering + epsilon)

    # Composite score calculation
    score = (
        forecasted_mean_normalized
        - (risk_percentage * rachev_normalized)
        + sharpe_normalized
        + sortino_normalized
        - volatility_clustering_normalized
    )

    return score

def smooth_scores(scores, alpha=0.2):
    """
    Smooth scores using Exponential Moving Average (EMA).
    """
    if not isinstance(scores, (list, np.ndarray)):
        raise ValueError("Scores must be a list or numpy array of numerical values.")
    
    scores = np.array(scores)
    smoothed_scores = np.zeros_like(scores)
    smoothed_scores[0] = scores[0]  # Initialize EMA
    
    for t in range(1, len(scores)):
        smoothed_scores[t] = alpha * scores[t] + (1 - alpha) * smoothed_scores[t - 1]
    
    return smoothed_scores.tolist()

def calculate_means_and_stds_monthly(etf_pred_dict, forecast_period):
    returns_list = [etf_pred_dict[etf][f'returns_{forecast_period}'] for etf in etf_pred_dict]
    mean_forecast = np.mean([np.mean(returns) for returns in returns_list])
    std_forecast = np.std([np.mean(returns) for returns in returns_list])

    rachev_ratios = np.array([etf_pred_dict[etf][f'rachev_ratio_{forecast_period}'] for etf in etf_pred_dict])
    mean_rachev = np.mean(rachev_ratios)
    std_rachev = np.std(rachev_ratios)

    sharpe_ratios = np.array([etf_pred_dict[etf][f'sharpe_ratio_{forecast_period}'] for etf in etf_pred_dict])
    mean_sharpe = np.mean(sharpe_ratios)
    std_sharpe = np.std(sharpe_ratios)

    sortino_ratios = np.array([etf_pred_dict[etf][f'sortino_ratio_{forecast_period}'] for etf in etf_pred_dict])
    mean_sortino = np.mean(sortino_ratios)
    std_sortino = np.std(sortino_ratios)

    volatility_clustering = np.array([etf_pred_dict[etf][f'volatility_clustering_{forecast_period}'] for etf in etf_pred_dict])
    mean_volatility_clustering = np.mean(volatility_clustering)
    std_volatility_clustering = np.std(volatility_clustering)

    return (
        mean_forecast, std_forecast, mean_rachev, std_rachev,
        mean_sharpe, std_sharpe, mean_sortino, std_sortino,
        mean_volatility_clustering, std_volatility_clustering
    )

def calculate_scores_for_etfs_monthly(etf_pred_dict, forecast_period, risk_percentage, alpha=0.2):
    (mean_forecast, std_forecast, mean_rachev, std_rachev, 
     mean_sharpe, std_sharpe, mean_sortino, std_sortino, 
     mean_volatility_clustering, std_volatility_clustering) = calculate_means_and_stds_monthly(etf_pred_dict, forecast_period)

    raw_scores = []
    for etf in etf_pred_dict:
        forecasted_values = etf_pred_dict[etf][f'returns_{forecast_period}']
        rachev_ratio = etf_pred_dict[etf][f'rachev_ratio_{forecast_period}']
        sharpe_ratio = etf_pred_dict[etf][f'sharpe_ratio_{forecast_period}']
        volatility_clustering = etf_pred_dict[etf][f'volatility_clustering_{forecast_period}']
        sortino_ratio = etf_pred_dict[etf][f'sortino_ratio_{forecast_period}']

        score = calculate_composite_score(
            forecasted_values, risk_percentage, rachev_ratio, sharpe_ratio, 
            sortino_ratio, volatility_clustering, mean_forecast, std_forecast, 
            mean_rachev, std_rachev, mean_sharpe, std_sharpe, mean_sortino, 
            std_sortino, mean_volatility_clustering, std_volatility_clustering
        )

        raw_scores.append(score)

    # Apply smoothing to the raw scores
    smoothed_scores = smooth_scores(raw_scores, alpha=alpha)

    results = []
    for idx, etf in enumerate(etf_pred_dict):
        results.append({
            'ETF': etf,
            'Month': forecast_period,
            'RiskPercentage': risk_percentage,
            'RawScore': raw_scores[idx],
            'SmoothedScore': smoothed_scores[idx]
        })

    return results

def main_monthly(tickers, etf_dict):
    etf_pred_dict = process_etf_data_monthly(tickers, etf_dict)
    risk_percentage = 0.10
    monthly_scores = {}

    for month in range(1, 13):
        month_key = f"{month}m"
        if any(f"returns_{month_key}" in etf_pred_dict[etf] for etf in etf_pred_dict):
            scores = calculate_scores_for_etfs_monthly(etf_pred_dict, month_key, risk_percentage)
            monthly_scores[month_key] = scores
        else:
            print(f"Skipping score calculation for {month_key}: No ETFs have data for this month.")

    return etf_pred_dict, monthly_scores

# Select top ETFs for monthly portfolio management
def select_top_etfs_monthly(df_scores, forecast_period):
    if df_scores.empty:
        print(f"No scores available for {forecast_period}. Skipping.")
        return []
    top_etfs = df_scores.nlargest(2, 'SmoothedScore')
    return top_etfs['ETF'].tolist()

etf_pred_dict, monthly_scores = main_monthly(tickers, etf_dict)


# Initialize an empty dictionary to hold DataFrames for each month
monthly_scores_dfs = {}

# Assuming monthly_scores is a dictionary with keys as month identifiers ('1m', '2m', ..., '12m') 
# and values as the respective scores
for month_key, scores in monthly_scores.items():
    # Convert scores for the current month into a DataFrame
    monthly_scores_dfs[month_key] = pd.DataFrame(scores)

    
# Create DataFrame variables dynamically for 12 months
for month in range(1, 13):
    month_key = f'{month}m'
    globals()[f'df_scores_{month_key}'] = monthly_scores_dfs.get(month_key, pd.DataFrame())


# Initialize a dictionary to hold the top 2 ETFs for each month
top_etfs_monthly = {}

# Loop through the monthly DataFrames and select the top 2 ETFs for each month
for month_key, df_scores in monthly_scores_dfs.items():
    # Select the top 2 ETFs based on the 'Score' column
    top_etfs_monthly[month_key] = df_scores.nlargest(2, 'SmoothedScore')
    
    # Print the results for the current month
    print(f"\nTop 2 ETFs for {month_key} forecast:")
    print(top_etfs_monthly[month_key])



Debug: Returns for SMH, Forecast Period 1m: [-0.03424207 -0.0177656  -0.00613295  0.00685085  0.03585814  0.00294293
  0.00110846  0.00592081 -0.00325445  0.01588209 -0.0060796   0.03215366
  0.03768135  0.00474041  0.00557297  0.0177254   0.00030171 -0.02152142
  0.01216641 -0.00953063 -0.0142531 ]

Debug: Returns for SMH, Forecast Period 2m: [ 0.00914876  0.01957979  0.01957979 -0.01109389  0.0199053   0.01432536
  0.022254   -0.0037174  -0.01843396  0.0199053  -0.00325445 -0.00413795
 -0.01976739 -0.00620238  0.06652507 -0.00613295  0.00850707 -0.00252586
 -0.01108421  0.02176784]

Debug: Returns for SMH, Forecast Period 3m: [ 4.0259127e-02  1.6403226e-02 -1.4875734e-02  2.5091643e-02
  3.4783494e-02 -3.9856419e-02 -1.7832287e-02  3.1111727e-02
 -1.9767394e-02 -1.7912013e-02 -6.9202459e-03  2.1317771e-03
 -2.0153981e-03  1.6323499e-02  2.2416493e-02  6.1173928e-03
 -2.2397200e-03 -9.1863824e-03 -8.0723556e-05  8.6368917e-04]

Debug: Returns for SMH, Forecast Period 4m: [ 0.01366763

In [48]:
# Generate Month Ranges
month_ranges = generate_month_ranges('2024-01-01', '2024-12-01')

# Assuming `tickers` is a list of ETF tickers
tickers = tickers

# Gather ETF Data
etf_histories = gather_etf_data_for_months(tickers, month_ranges)

# Create a mapping between month numbers and date ranges
month_key_mapping = {f"{i+1}m": month_range for i, month_range in enumerate(etf_histories.keys())}

# Debug: Print the month key mapping
print("Month Key Mapping:", month_key_mapping)

# Align `top_etfs_monthly` Keys with `etf_histories`
aligned_top_etfs_monthly = {}

for month_key, df_scores in monthly_scores_dfs.items():
    # Call the `select_top_etfs_monthly` function here
    forecast_period = month_key_mapping.get(month_key, None)
    if forecast_period:
        aligned_top_etfs_monthly[forecast_period] = select_top_etfs_monthly(df_scores, forecast_period)

# Portfolio initialization and management
ticker_shares = {}
ticker_shares_per_month = {}

for i, month_range in enumerate(etf_histories.keys()):
    current_month_key = month_range
    if i == 0:
        # First month initialization
        ticker_shares = initialize_shares_for_first_month(
            aligned_top_etfs_monthly[current_month_key],
            etf_histories,
            current_month_key
        )
        ticker_shares_per_month[current_month_key] = ticker_shares.copy()
    else:
        # Subsequent months' portfolio management
        prev_month_key = list(etf_histories.keys())[i - 1]
        ticker_shares = manage_etf_portfolio_monthly(
            aligned_top_etfs_monthly[prev_month_key],
            aligned_top_etfs_monthly[current_month_key],
            prev_month_key,
            current_month_key,
            ticker_shares,
            etf_histories
        )
        ticker_shares_per_month[current_month_key] = ticker_shares.copy()

print("\nFinal Ticker Shares per Month:")
for month, shares in ticker_shares_per_month.items():
    print(f"Month {month}: {shares}")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Month Key Mapping: {'1m': '2024-01-01 to 2024-01-31', '2m': '2024-02-01 to 2024-02-29', '3m': '2024-03-01 to 2024-03-31', '4m': '2024-04-01 to 2024-04-30', '5m': '2024-05-01 to 2024-05-31', '6m': '2024-06-01 to 2024-06-30', '7m': '2024-07-01 to 2024-07-31', '8m': '2024-08-01 to 2024-08-31', '9m': '2024-09-01 to 2024-09-30', '10m': '2024-10-01 to 2024-10-31', '11m': '2024-11-01 to 2024-11-30'}
Shares 1st month: (50000 * 0.975) / 139.7899932861328
Ticker: FTEC, First trading day: 2024-01-02, Price: 139.7899932861328, Shares: 348.74
Shares 1st month: (50000 * 0.975) / 72.7249984741211
Ticker: IGM, First trading day: 2024-01-02, Price: 72.7249984741211, Shares: 670.33
Top 2 ETFs for 2024-01-01 to 2024-01-31: ['FTEC', 'IGM']
Top 2 ETFs for 2024-02-01 to 2024-02-29: ['SMH', 'PSI']
Formula:  348.73740855123145  *  148.61000061035156 * 0.975
Sell FTEC: 348.74 shares at 148.61. Total value: 50530.22  

Formula:  670.3334619848428  *  78.93000030517578 * 0.975
Sell IGM: 670.33 shares at 78.93. T

In [49]:
# Define the first trading day of December 2024
first_trading_day_dec = '2024-12-01'  # Adjust this to match the actual start date of December

# Identify the November and December month date range keys
november_range = list(ticker_shares_per_month.keys())[-1]  # Last key corresponds to November
december_start = first_trading_day_dec  # Replace with the actual start of December
print(f"Using data for November: {november_range}")

print(f"Fetching data starting from the first trading day of December: {december_start}")

december_end = '2024-12-31'
# Initialize a dictionary to store the values of shares
etf_values_dec = {}

# Ensure November data exists
if november_range in ticker_shares_per_month:
    # Fetch ETF shares from November
    ticker_shares_nov = ticker_shares_per_month[november_range]
    
    # Fetch the first trading day price of December for each ETF
    for ticker, shares in ticker_shares_nov.items():
        print(f"Fetching data for ticker {ticker} starting from {december_start}...")
        # Download historical data for December
        data = yf.download(ticker, start=december_start, end=december_end)
        
        if not data.empty:
            # Get the closing price of the first trading day of December
            closing_price_dec = data['Close'].iloc[0]
            # Calculate the value of the shares
            total_value = shares * closing_price_dec
            etf_values_dec[ticker] = total_value
            print(f"{ticker}: {shares:.2f} shares at ${closing_price_dec:.2f} each, total value: ${total_value:.2f}")
        else:
            print(f"{ticker}: No data available for December's first trading day.")
else:
    print(f"No data available in ticker_shares_per_month for November: {november_range}")



[*********************100%%**********************]  1 of 1 completed


Using data for November: 2024-11-01 to 2024-11-30
Fetching data starting from the first trading day of December: 2024-12-01
Fetching data for ticker FTEC starting from 2024-12-01...
FTEC: 1400.89 shares at $186.99 each, total value: $261952.88
Fetching data for ticker IGM starting from 2024-12-01...


[*********************100%%**********************]  1 of 1 completed

IGM: 2495.37 shares at $102.51 each, total value: $255800.47


In [50]:
# Check if there are any values in the dictionary
if etf_values_dec:
    print("\nETF values on December's first trading day:")
    # Sum all the values (total portfolio value)
    total_value = sum(etf_values_dec.values())
    print(f"Total portfolio value: {total_value:.2f}")
    # Optionally display individual ETF values
    for ticker, value in etf_values_dec.items():
        print(f"{ticker}: {value:.2f}")
else:
    print("No values could be calculated for December's first trading day.")



ETF values on December's first trading day:
Total portfolio value: 517753.35
FTEC: 261952.88
IGM: 255800.47
